# Paprika Database

This script connects to Paprika app's SQLite database and pull out whatever we want and format it as JSON, YAML, whatever.

## NOTE:
Before running the first time, you must create a file in the same directory as this Notebook, named "config.py"
and it set this var:

path_project    = "/local/path/to/this/repo/joi.github.io"

_(which should be the path to the direcotry one level up from where this file here is.)_

In [562]:
import os
import datetime
from datetime import datetime
from zipfile import ZipFile
import json
import sqlite3
from sqlite3 import Error
from pathlib import Path
from shutil import copyfile
import config # This imports our local config file, "config.py". Access vars like so: config.var

In [563]:
# Vars!

# Date time stamp
now = datetime.now()
dt = now.strftime("%Y-%m-%d %H.%M")
#print(dt)

# Environment-dependent Paths
path_project    = config.path_project # manually set in config.py, which is NOT checked into GIT.
path_user_home  = str(Path.home())    # automatically detects the User home directory path from the OS/ENV

# Paprika Database Path
path_db_sub     = '/Library/Group Containers/72KVKW69K8.com.hindsightlabs.paprika.mac.v3/Data/Database/'
filename_db     = 'Paprika.sqlite'
path_db_full    = path_user_home + path_db_sub + filename_db

# Working file path. / We create this to work on and then delete it when done.
file_db_working     = 'Paprika-working.sqlite'
path_db_working     = path_user_home + path_db_sub + file_db_working
path_db_shm         = path_user_home + path_db_sub + file_db_working + "-shm"
path_db_wal         = path_user_home + path_db_sub + file_db_working + "-wal"

# Paprika Databse Backup. / We create this before we do anything, just in case.
file_db_bu          = 'Paprika-BU-' + dt + '.sqlite'
path_db_bu_sub      = path_project + '/__scripts/backups/'
path_db_bu          = path_db_bu_sub + file_db_bu + '.zip'

# Only needed once really…
if not os.path.exists(path_db_bu_sub):
    os.mkdir(path_db_bu_sub)

# SET SCRIPT VARIABLES

myVars = {
    'paths': {
        'paprika_export':   './',
        'paprika_jekyll':   '../_recipes/',
        'paprika_jsondump': '../_data/recipes/',
        'jekyll_data': '../_data/',
        'jekyll_root': '../'
    },
    'sources': {
        'paprika_export': 'data/My Recipes.paprikarecipes',
        'paprika_h_zip' : 'data/My Recipes.zip',
        'paprika_html'  : 'data/My Recipes/Recipes/'
    }
}
# Learning to Pretty Print DICTs. I like to see the data I am sculpting with
#pp = pprint.PrettyPrinter(indent=4)
#pp.pprint(myVars)


In [564]:
# UTILITY FUNCTIONS

# Clobber a string into a filename -------------
def make_filename(string):
    string = unidecode.unidecode(string)
    string = string.replace(" ","-")
    #string = created[0:10] + "-" + string
    #string=str(bytes(string, 'utf-8').decode('utf-8','ignore').encode("utf-8",'ignore'))
    #string=string.replace("b'","").replace("'","")
    invalid = '<>:"/\|?* ,()“”‘’\''
    for char in invalid:
        string = string.replace(char, '')
    string = sanitize_filename(string)
    
    return string



In [565]:
# Make a zipped backup of the DB
zipfile.ZipFile(path_db_bu, mode='w').write(path_db_full, arcname=file_db_bu, compress_type=zipfile.ZIP_DEFLATED, compresslevel=9)

# Make a copy of the DB to work with. We delete it later.
copy_db = copyfile(path_db_full, path_db_working) # create a var here just to capture the useless out put of the copyfile() function

In [566]:
# DATABASE FUNCTIONS

# Connect to Database
def db_connect(db_file):
    #conn = None
    try:
        conn = sqlite3.connect(db_file)
        # row_factory does some magic for us
        # see: https://stackoverflow.com/questions/3300464/how-can-i-get-dict-from-sqlite-query
        conn.row_factory = sqlite3.Row
    except Error as e:
        print(e)
    return conn

In [567]:
# Open Connection to Database
conn = db_connect(path_db_working)

# First Request: Get our recipe Data
with conn:
    cur = conn.cursor()
    cur.execute("""
SELECT 
    `ZCOOKTIME`        as `cook_time`,
    `ZCREATED`         as `created`,
    `ZDESCRIPTIONTEXT` as `description`,
    `ZDIFFICULTY`      as `difficulty`,
    `ZDIRECTIONS`      as `directions`,
    `ZINGREDIENTS`     as `ingredients`,
    `ZNAME`            as `name`,
    `ZNOTES`           as `notes`,
    `ZNUTRITIONALINFO` as `nutritional_info`,
    `ZPHOTO`           as `photo`,
    `ZPREPTIME`        as `prep_time`,
    `ZSCALE`           as `rating`,
    `ZSERVINGS`        as `servings`,
    `ZSOURCE`          as `source`,
    `ZSOURCEURL`       as `source_url`,
    `ZTOTALTIME`       as `total_time`,
    `ZUID`             as `uid`
FROM
    ZRECIPE
"""
    )

#For the next bit with columns and results and dict and zip, see:
#    https://stackoverflow.com/questions/16519385/output-pyodbc-cursor-results-as-python-dictionary/16523148#16523148

# This grabs the key (cur.descriptiomn) for us
columns = [column[0] for column in cur.description]
rows = cur.fetchall()

results = []
for row in rows:
    # and here we glue the key to the value
    results.append(dict(zip(columns, row)))


# Convert the Results to JSON
json_output = json.dumps(results, ensure_ascii=False, indent=4, sort_keys=True)

f = open('output.json', 'w')
f.write(json_output)
f.close()    
#print(json_output)



#Cleanup
os.unlink(path_db_working)
os.unlink(path_db_shm)
os.unlink(path_db_wal)